In [1]:
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'C:\Users\Karen Fernandes\anaconda3\Files\Projects\Flipkart API\data_extraction_phone_dataset.csv')
df.head(2)

,Unnamed: 0.1,Unnamed: 0,pid,itemId,listingId,brand,title,subTitle,url,badge,...,screen_size_cm,screen_size_inch,display,ram,storage,expandable_storage,color,processor,battery,average_rating
0,0,0,MOBGMFFX5XYE8MZN,ITMC77FF94CDF044,LSTMOBGMFFX5XYE8MZNRGKCA5,SAMSUNG,"SAMSUNG Galaxy S23 5G (Cream, 128 GB)",8 GB RAM,https://flipkart.com/samsung-galaxy-s23-5g-cre...,NaN,...,15.49,6.10,Full HD+,8 GB RAM,128 GB,NaN,Cream,"', 'Qualcomm Snapdragon 8 Gen 2 Processor",3900 mAh Lithium Ion Battery,4.5
1,1,1,MOBHY9PQMNCMDVCD,ITM1C4B849213A0E,LSTMOBHY9PQMNCMDVCDI6UUHQ,MOTOROLA,"Motorola G85 5G (Olive Green, 128 GB)",8 GB RAM,https://flipkart.com/motorola-g85-5g-olive-gre...,NaN,...,16.94,6.67,Full HD+,8 GB RAM,128 GB,NaN,Olive Green,"', '6s Gen 3 Processor",5000 mAh Battery,4.4


In [3]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'pid', 'itemId', 'listingId', 'brand',
       'title', 'subTitle', 'url', 'badge', 'stock', 'highlights',
       'isSponsored', 'mrp', 'price', 'rating', 'images', 'model', 'camera',
       'screen_size_cm', 'screen_size_inch', 'display', 'ram', 'storage',
       'expandable_storage', 'color', 'processor', 'battery',
       'average_rating'],
      dtype='object')

In [4]:
# drop Unnamed: 0 column
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

In [5]:
df1 = df[['pid','brand', 'title', 'subTitle', 'stock', 'highlights',
       'mrp', 'price', 'rating', 'images', 'model', 'camera',
       'screen_size_cm', 'screen_size_inch', 'display', 'ram', 'storage',
       'color', 'processor', 'battery', 'average_rating']]

In [6]:
# convert the title names to proper case
df1['title'] = df1['title'].str.title()

In [7]:
# convert the brand names to proper case
df1['brand'] = df1['brand'].str.title()

In [8]:
# convert the model names to proper case
df1['model'] = df1['model'].str.title()

In [9]:
# extract rear camera
df1['rear_camera'] = df1['camera'].str.extract(r'(\d+MP) Rear Camera')
df1['rear_camera'] = df1['rear_camera'].fillna(df1['camera'].str.extract(r'(\d+MP)').iloc[:, 0])

In [10]:
# extract front camera
df1['front_camera'] = df1['camera'].str.extract(r'(\d+MP) Front Camera')
df1['front_camera'] = df1['front_camera'].fillna(df1['camera'].str.extract(r'\+ (\d+MP)').iloc[:, 0])

In [11]:
# clearing out 'MP' in front_camera and rear_camera
df1['front_camera'] = df1['front_camera'].str.replace(r'MP', '', regex=True)
df1['rear_camera'] = df1['rear_camera'].str.replace(r'MP', '', regex=True)

In [12]:
df1['processor'] = df1['processor'].str.replace("', '", "")

In [13]:
# clearing out 'RAM'
df1['ram'] = df1['ram'].str.replace(r' GB RAM', '', regex=True)

In [14]:
df1 = df1[~df1['ram'].str.contains('MB RAM', na=False)]

In [15]:
# clearing out 'GB' in storage
df1['storage'] = df1['storage'].str.replace(r' GB', '', regex=True)

In [16]:
# Regex pattern to match RAM specifications
ram_pattern = r'\b\d+\s*(GB|MB)\s*RAM\b'

# Replace RAM specifications with an empty string and strip any leading/trailing whitespace
df1['subTitle'] = df1['subTitle'].str.replace(ram_pattern, '', regex=True).str.strip()

# Transfer values from 'subTitle' to 'color' where 'color' is NaN
df1['color'] = df1['color'].fillna(df1['subTitle'])

# convert the brand names to proper case
df1['color'] = df1['color'].str.title()

In [17]:
# This regex assumes the color name is the last part after a closing parenthesis or comma
df1['color'] = df1['color'].str.extract(r'\b(\w+(?: \w+)*?)\s*$')

# Handle cases where the color might be not properly extracted
df1['color'] = df1['color'].fillna(df1['color'].str.strip())

In [18]:
# drop subTitle column
df1.drop(columns=['subTitle'], inplace=True)

In [19]:
df2 = df1.dropna(subset=['mrp', 'price', 'screen_size_cm', 'screen_size_inch', 'color'])

In [20]:
# Define a function to update processor information
def extract_processor(model):
    if pd.isna(model):
        return None
    if re.search('SAMSUNG Galaxy M14 4G', model, re.IGNORECASE): 
        return 'Snapdragon 680 Processor'
    elif re.search('SAMSUNG Galaxy M34 5G', model, re.IGNORECASE): 
        return 'Exynos 1280 Processor'
    elif re.search('SAMSUNG Galaxy M35 5G', model, re.IGNORECASE): 
        return 'Exynos Processor'
    elif re.search('SAMSUNG Galaxy M14 5G', model, re.IGNORECASE): 
        return 'Exynos Processor'
    elif re.search('Samsung Galaxy M13 5G', model, re.IGNORECASE): 
        return 'Mediatek Processor'
    elif re.search('OnePlus Nord CE 3 Lite 5G', model, re.IGNORECASE): 
        return 'Qualcomm Snapdragon 695 Processor'
    elif re.search('Realme Narzo N63 4G', model, re.IGNORECASE): 
        return 'Mediatek Processor'
    elif re.search('Realme Narzo N63', model, re.IGNORECASE): 
        return 'Unisoc Processor'
    elif re.search('Redmi 13C 5G', model, re.IGNORECASE): 
        return 'Mediatek Processor'
    else:
        return None

# Apply the function to update the 'processor' column
df2['processor'] = df2.apply(lambda row: extract_processor(row['model']) if pd.isna(row['processor']) else row['processor'], axis=1)

In [21]:
# no brand of NA Processor available
df2['processor'] = df2['processor'].str.replace(r'NA Processor','', regex = True)

In [22]:
# Define a function to update battery information
def extract_battery(model):
    if pd.isna(model):
        return None
    if re.search('Apple IPhone 15 Plus', model, re.IGNORECASE):  
        return '4383 mAh Battery'
    elif re.search('Apple IPhone 15', model, re.IGNORECASE): 
        return '3349 mAh Battery'
    elif re.search('Apple IPhone 13', model, re.IGNORECASE): 
        return '3240 mAh Battery'
    elif re.search('Apple IPhone 11', model, re.IGNORECASE): 
        return '3110 mAh Battery'
    elif re.search('Apple IPhone 12', model, re.IGNORECASE): 
        return '3100 mAh Battery'
    elif re.search('Apple IPhone 14', model, re.IGNORECASE): 
        return '3279 mAh Battery'
    elif re.search('Apple IPhone 14 Plus', model, re.IGNORECASE): 
        return '4500 mAh Battery'
    elif re.search('Samsung Galaxy S21 FE 5G', model, re.IGNORECASE): 
        return '3279 mAh Battery'
    elif re.search('Motorola G32', model, re.IGNORECASE): 
        return '5000 mAh Battery'
    elif re.search('SAMSUNG Galaxy Z Flip3 5G', model, re.IGNORECASE): 
        return '3300 mAh Battery'
    else:
        return '5000 mAh Battery'

# Apply the function to update the 'battery' column
df2['battery'] = df2.apply(lambda row: extract_battery(row['model']) if pd.isna(row['battery']) else row['battery'], axis=1)

In [23]:
# Extract the numeric part
df2['battery'] = df2['battery'].str.extract('(\d+)').astype(int)

In [24]:
# drop camera, screen_size_cm
df2.drop(columns=['camera', 'screen_size_cm'], inplace=True)

In [25]:
df2 = df2.dropna()

In [26]:
# Count NaN values
null_counts = df2.isnull().sum()
total = null_counts.sum()

print(f'null counts: {null_counts}')
print(f'total: {total}')

null counts: pid                 0
brand               0
title               0
stock               0
highlights          0
mrp                 0
price               0
rating              0
images              0
model               0
screen_size_inch    0
display             0
ram                 0
storage             0
color               0
processor           0
battery             0
average_rating      0
rear_camera         0
front_camera        0
dtype: int64
total: 0


In [27]:
df2.head()

,pid,brand,title,stock,highlights,mrp,price,rating,images,model,screen_size_inch,display,ram,storage,color,processor,battery,average_rating,rear_camera,front_camera
0,MOBGMFFX5XYE8MZN,Samsung,"Samsung Galaxy S23 5G (Cream, 128 Gb)",IN_STOCK,"['8 GB RAM | 128 GB ROM', '15.49 cm (6.1 inch)...",89999.0,49999.0,"{'average': 4.5, 'count': 13712, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...,Samsung Galaxy S23 5G,6.10,Full HD+,8,128,Cream,Qualcomm Snapdragon 8 Gen 2 Processor,3900,4.5,50,12
1,MOBHY9PQMNCMDVCD,Motorola,"Motorola G85 5G (Olive Green, 128 Gb)",IN_STOCK,"['8 GB RAM | 128 GB ROM', '16.94 cm (6.67 inch...",20999.0,17999.0,"{'average': 4.4, 'count': 19290, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...,Motorola G85 5G,6.67,Full HD+,8,128,Olive Green,6s Gen 3 Processor,5000,4.4,50,32
5,MOBGY2JUGGJZJQNK,Motorola,"Motorola G64 5G (Ice Lilac, 256 Gb)",IN_STOCK,['12 GB RAM | 256 GB ROM | Expandable Upto 1 T...,19999.0,16999.0,"{'average': 4.2, 'count': 16352, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...,Motorola G64 5G,6.50,Full HD+,12,256,Ice Lilac,Dimensity 7025 Processor,6000,4.2,50,16
6,MOBGY2JUPSSWCZEJ,Motorola,"Motorola G64 5G (Mint Green, 256 Gb)",IN_STOCK,['12 GB RAM | 256 GB ROM | Expandable Upto 1 T...,19999.0,16999.0,"{'average': 4.2, 'count': 16352, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...,Motorola G64 5G,6.50,Full HD+,12,256,Mint Green,Dimensity 7025 Processor,6000,4.2,50,16
7,MOBHY9PQZACQBKGG,Motorola,"Motorola G04S (Sea Green, 64 Gb)",IN_STOCK,['4 GB RAM | 64 GB ROM | Expandable Upto 1 TB'...,9999.0,6999.0,"{'average': 4.2, 'count': 13447, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...,Motorola G04S,6.60,HD+,4,64,Sea Green,T606 Processor,5000,4.2,50,5


In [28]:
df2.shape

(525, 20)

In [29]:
# renaming column
df2.to_csv('cleaned_phones_dataset.csv')